In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import matplotlib.pyplot as plt
import numpy as np
import math

In [3]:
def _bn_relu(inputs):
    x = tf.keras.layers.BatchNormalization()(inputs)       #BatchNorm
    x = tf.keras.layers.Activation('relu')(x)              #relu
    return x


def _conv(inputs, filters, kernel_size, strides=1, padding='same',
          kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1.e-4)):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding=padding,
               kernel_initializer=kernel_initializer,
               kernel_regularizer=kernel_regularizer)(inputs)
    return x


def _bn_relu_conv(inputs, filters, kernel_size, strides=1, padding='same',
                  kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1.e-4)):
    # pre activation
    x = _bn_relu(inputs)
    x = _conv(x, filters, kernel_size, strides=strides, padding=padding,
              kernel_initializer=kernel_initializer,
              kernel_regularizer=kernel_regularizer)
    return x


def _conv_bn_relu(inputs, filters, kernel_size, strides=1, padding='same',
                  kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1.e-4)):
    x = _conv(inputs, filters, kernel_size, strides=strides, padding=padding,
              kernel_initializer=kernel_initializer,
              kernel_regularizer=kernel_regularizer)
    x = _bn_relu(x)
    return x


def bottleneck(inputs, filters, strides=1):
    x = _conv(inputs, filters, (1, 1), strides=strides, padding='valid')
    return x


def conv_block(inputs, filters, rep, is_first=False):
    shortcut = bottleneck(inputs, filters * 4, strides=(2 if not is_first else 1))
    shortcut = tf.keras.layers.BatchNormalization()(shortcut)

    x = inputs

    for i in range(rep):
        x = bottleneck(x, filters, strides=(2 if not is_first and i == 0 else 1))            
        x = _bn_relu_conv(x, filters, (3, 3))
        x = _bn_relu_conv(x, filters * 4, (1, 1), padding='valid')
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Add()([x, shortcut])
        x = tf.keras.layers.Activation('relu')(x)

        shortcut = x
    return x

In [5]:
def ResNet50():
    inputs = tf.keras.layers.Input(shape=(224,224,3))
    x = _conv_bn_relu(inputs, 64, (7,7), strides=2, padding='same')
    x = tf.keras.layers.MaxPool2D((3,3), strides=2)(x)

    x = conv_block(x, 64, 3, True)
    x = conv_block(x, 128, 4)
    x = conv_block(x, 256, 6)
    x = conv_block(x, 512, 3)

    x = tf.keras.layers.GlobalAvgPool2D()(x)
    outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    return model

In [6]:
resnet = ResNet50()
resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

resnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________